In [1]:
import requests
import json
import time

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import load_model

from watson_developer_cloud import ToneAnalyzerV3
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, EntitiesOptions, KeywordsOptions
import numpy as np


Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#Code for speech to text conversion using RevSpeech API

API_KEY = "Bearer 01GQPjLQxzjnRB1lkzQ7ptri6wxAO1CYUbkZ3jkJiHZGoyYhT1csCeFNa-FW1aDKBS3HVUTTvWmmGdDyd6XP_A7zBeHZM"
HEADERS = {'Authorization': API_KEY}

def submit_job_url(media_url):
    url = "https://api.rev.ai/revspeech/v1beta/jobs"
    payload = {'media_url': media_url,
               'metadata': "Test"}
    request = requests.post(url, headers=HEADERS, json=payload)

    if request.status_code != 200:
        raise

    response_body = request.json()
    return response_body['id']

def submit_job_file(file):
    url = "https://api.rev.ai/revspeech/v1beta/jobs"
    files = { 'media': (file, open(file, 'rb'), 'audio/mp3') }
    request = requests.post(url, headers=HEADERS, files=files)
    if request.status_code != 200:
        raise

    response_body = request.json()
    return response_body['id']

def view_job(id=59594172):
    url = f'https://api.rev.ai/revspeech/v1beta/jobs/{id}'
    request = requests.get(url, headers=HEADERS)

    if request.status_code != 200:
        raise

    response_body = request.json()
    return response_body

def get_transcript(id='59594172'):
    url = f'https://api.rev.ai/revspeech/v1beta/jobs/{id}/transcript'
    headers = HEADERS.copy()
    headers['Accept'] = 'application/vnd.rev.transcript.v1.0+json'
    request = requests.get(url, headers=headers)

    if request.status_code != 200:
        raise

    response_body = request.json()
    return response_body

def test_workflow_with_url(url):
    print ("Submitting job with URL")
    id = submit_job_url(url)
    print ("Job created")
    view_job(id)

    while True:
        job = view_job(id)
        status = job["status"]
        print (f'Checking job transcription status: { status }')
        if status == "transcribed":
            break
        if status == "failed":
            raise

        print ("Trying in another 30 seconds")
        time.sleep(30)

    return get_transcript(id)

def test_workflow_with_file(file):
    print ("Submitting job with file")
    id = submit_job_file(file)
    print ("Job created")
    view_job(id)

    while True:
        job = view_job(id)
        status = job["status"]
        print (f'Checking job transcription status: { status }')
        if status == "transcribed":
            break
        if status == "failed":
            raise

        print ("Trying in another 30 seconds")
        time.sleep(30)

    return get_transcript(id)

file = "test.mp3"
print(test_workflow_with_file(file))

Submitting job with file
Job created
Checking job transcription status: in_progress
Trying in another 30 seconds
Checking job transcription status: in_progress
Trying in another 30 seconds
Checking job transcription status: transcribed
{'monologues': [{'speaker': 0, 'elements': [{'type': 'text', 'value': 'Hello', 'ts': 0.8099999999999999, 'end_ts': 1.14, 'confidence': 1.0}, {'type': 'punct', 'value': ', '}, {'type': 'text', 'value': 'this', 'ts': 1.14, 'end_ts': 1.38, 'confidence': 1.0}, {'type': 'punct', 'value': ' '}, {'type': 'text', 'value': 'is', 'ts': 1.38, 'end_ts': 1.5, 'confidence': 1.0}, {'type': 'punct', 'value': ' '}, {'type': 'text', 'value': 'a', 'ts': 1.5, 'end_ts': 1.5899999999999999, 'confidence': 0.98}, {'type': 'punct', 'value': ' '}, {'type': 'text', 'value': 'test', 'ts': 1.5899999999999999, 'end_ts': 2.2199999999999998, 'confidence': 1.0}, {'type': 'punct', 'value': '.'}]}]}


In [3]:
#List of manually created datapoints for urgency rating algorithm.
urgencyList = {
    "The electricity in my neighborhood is out. Can you bring it back up? I just want power." : 1,
    "This is my luckiest day ever. I am so happy. I just ate a cookie. It was delicious. " : 1,
    "Can you bring me tasty food? I want to enjoy my 18\'th birthday. I move to college next week, so it would be great if you could do this." : 1,
    "My son has a mild headache. Can you help me? I am worried about his health, and he is only 5 years old." : 1,
    "I feel slightly nauseous after drinking coffee for the first time in a long time. What medicines should I take?" : 1,
    "I fell and scraped my knee. Now I am bleeding and I'm worried about a bacterial infection." : 2,
    "My daughter just broke her pinky finger and it hurts her quite a bit. I think she will be okay, but can I get medical assistance." : 2,
    "I tripped and had a concussion at work, and it hurts a lot. I feel like vomiting. I need help soon, or I will faint." : 2,
    "My grandfather is coughing a lot. I think he has the flu and I\'m worried for his health. We don\'t have any medicine" : 2,
    "A tree just fell and destroyed our entire house. This is horrible. I\'ve lived here for 20 years, and now it's all gone?" : 2,
    "The water supply is contaminated. We don\'t have any clean water. Please bring us water immediately, or we may die. " : 3,
    "A tree just fell on my husband\'s back. I can\'t even hear his heartbeat anymore. Oh god, please help immediately, before he dies." : 3,
    "My father just had a heart attack. He's writhing in pain, I've never seen him suffer like this before. Please, this is extremely urgent. He could die!" : 3,
    "I just found both of my parents dead in the living room. Oh god, there's blood everywhere. This can't be real. How could anybody do this? This is the worst thing I've ever seen." : 3,
    "Please, help, this is extremely urgent. A big boulder just trapped ten of us in a cave, and we can't escape. If we don't get help immediately, we will soon die from lack of oxygen." : 3,
    }

#IBM Natural Language Understanding API
nlu = NaturalLanguageUnderstandingV1(version='2018-09-15',username='3298b4bc-e9a7-4e26-accb-5969030597d4',password='XRR7ex4OCw2q')

In [4]:
def get_vector(sentence):
    #returns a feature vector from the emotional decomposition(sadness, joy, fear, disgust, anger, sentiment) of a sentence.
    response = nlu.analyze(text=sentence,features=Features(entities=EntitiesOptions(emotion=True,sentiment=True,limit=2), keywords=KeywordsOptions(emotion=True,sentiment=True,limit=2))).get_result()
    accept = ['sadness', 'joy','fear','disgust','anger']
    vec = np.zeros(6)
    cnt = 0
    for g in response['keywords']:
        
        if('emotion' in g):
            cnt+=1
            for i in range(5):
                vec[i] += g['emotion'][accept[i]]
            vec[5] += g['sentiment']['score']

    for i in range(6):
        vec[i]/=cnt
    
    return vec

In [11]:
#build a neural network for evaluating urgency and saves it
def build_neural_model():
    trainx = np.zeros( (len(urgencyList),6) )
    trainy = np.zeros( (len(urgencyList),1) )
    ptr = 0
    for i in urgencyList:
        trainx[ptr,:] = get_vector(i)
        trainy[ptr] = urgencyList[i]
        ptr+=1

    model = Sequential()
    model.add(Dense(units=6, activation = 'relu', input_dim=6))
    model.add(Dense(units=4, activation = 'relu'))
    model.add(Dense(units=1, activation = 'linear'))
    model.compile(loss='mse', optimizer='adam')
    model.fit(trainx, trainy, epochs=750, verbose=0)
    model.save('neumodel.h5')

In [15]:
#speech to text translation
fileprefix = "test"

tests = []
for i in range(1,4):

    FullSet = test_workflow_with_file(fileprefix+str(i)+".mp3")['monologues'][0]['elements']
    this_string = ""
    for elem in FullSet:
        this_string += elem['value']
    tests.append(this_string)

print(tests)
    

Submitting job with file
Job created
Checking job transcription status: in_progress
Trying in another 30 seconds
Checking job transcription status: in_progress
Trying in another 30 seconds
Checking job transcription status: in_progress
Trying in another 30 seconds
Checking job transcription status: in_progress
Trying in another 30 seconds
Checking job transcription status: in_progress
Trying in another 30 seconds
Checking job transcription status: in_progress
Trying in another 30 seconds
Checking job transcription status: transcribed
Submitting job with file
Job created
Checking job transcription status: in_progress
Trying in another 30 seconds
Checking job transcription status: in_progress
Trying in another 30 seconds
Checking job transcription status: in_progress
Trying in another 30 seconds
Checking job transcription status: in_progress
Trying in another 30 seconds
Checking job transcription status: transcribed
Submitting job with file
Job created
Checking job transcription status: 

In [12]:
build_neural_model()

In [16]:
model = load_model('neumodel.h5')

ptr = 0
givenSents = tests
totx = np.zeros( (len(givenSents),6))
for i in givenSents:
    totx[ptr,:] = get_vector(i)
    ptr+=1
print(model.predict(totx))

[[2.8239572]
 [1.6484303]
 [2.2522426]]
